# PyTorch 3DResNet Classfication
This notebook has been converted to use PyTorch.

In [1]:
import os
import pandas as pd
import numpy as np
import nibabel as nib
from sklearn.model_selection import train_test_split
import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from sklearn.metrics import f1_score, accuracy_score

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"decive: {device}")

decive: cuda


In [3]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"


# 讀取數據

In [4]:
csv_path = 'structured_data_mac.csv'
df = pd.read_csv(csv_path)

# 檢查並清理數據

In [5]:
if df['Group'].isnull().any():
    df = df.dropna(subset=['Group'])

# 檢查是否存在非預期的標籤值

In [6]:
expected_labels = ['CN', 'MCI', 'AD']
df = df[df['Group'].isin(expected_labels)]

image_paths = df['Image Path'].tolist()

# 將標籤映射為數字

In [7]:
label_mapping = {'CN': 0, 'MCI': 1, 'AD': 2}
labels = df['Group'].map(label_mapping).tolist()

# 獲取獨特的病人ID

In [8]:
unique_patients = df['Subject'].unique()

# 根據病人ID進行訓練和驗證分割

In [9]:
train_ids, val_ids = train_test_split(unique_patients, test_size=0.2, random_state=42)

# 創建訓練和驗證DataFrame

In [10]:
train_df = df[df['Subject'].isin(train_ids)]
val_df = df[df['Subject'].isin(val_ids)]

# 提取訓練和驗證影像路徑和標籤

In [11]:
train_paths = train_df['Image Path'].tolist()
train_labels = train_df['Group'].map(label_mapping).tolist()
val_paths = val_df['Image Path'].tolist()
val_labels = val_df['Group'].map(label_mapping).tolist()

# 將標籤轉換為 tensor

In [12]:
train_labels_tensor = torch.tensor(train_labels, dtype=torch.long)
val_labels_tensor = torch.tensor(val_labels, dtype=torch.long)

In [13]:
def load_nifti_file(nifti_path):
    nifti_image = nib.load(nifti_path)
    return nifti_image.get_fdata()

In [14]:
def preprocess_image(image):
    processed_image = np.resize(image, (96, 96, 96))
    return processed_image

In [15]:
def data_generator(image_paths, labels, batch_size):
    while True:
        for i in range(0, len(image_paths), batch_size):
            batch_paths = image_paths[i:i + batch_size]
            batch_labels = labels[i:i + batch_size]

            batch_images = [preprocess_image(
                load_nifti_file(path)) for path in batch_paths]
            yield np.array(batch_images), np.array(batch_labels)

In [16]:
# Define the PyTorch dataset class
class NiftiDataset(Dataset):
    def __init__(self, image_paths, labels):
        self.image_paths = image_paths
        self.labels = labels

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        nifti_path = self.image_paths[idx]
        label = self.labels[idx]
        nifti_image = nib.load(nifti_path)
        image = nifti_image.get_fdata()
        image = np.resize(image, (96, 96, 96))
        image = torch.tensor(image, dtype=torch.float32)
        image = image.unsqueeze(0)  # Add channel dimension
        return image, label

In [17]:
# Create dataset instances
train_dataset = NiftiDataset(train_paths, train_labels_tensor)
val_dataset = NiftiDataset(val_paths, val_labels_tensor)

# Define DataLoaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

# 建立並編譯3D CNN模型

此段代碼顯示了如何建立並編譯一個用於二元分類的3D卷積神經網絡（CNN）模型。


In [18]:

import torch
import torch.nn as nn
import torch.nn.functional as F

# 定義 3D ResNet 模型
class ResNet3D(nn.Module):
    def __init__(self, block, layers, num_classes=3):
        self.in_channels = 64
        super(ResNet3D, self).__init__()
        self.conv1 = nn.Conv3d(1, 64, kernel_size=7, stride=(2, 2, 2), padding=(3, 3, 3), bias=False)
        self.bn1 = nn.BatchNorm3d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool3d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        self.avgpool = nn.AdaptiveAvgPool3d((1, 1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.in_channels != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv3d(self.in_channels, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm3d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.in_channels, planes, stride, downsample))
        self.in_channels = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.in_channels, planes))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x

# 定義模型的殘差塊
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_channels, channels, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv3d(in_channels, channels, kernel_size=3, stride=stride,
                               padding=1, bias=False)
        self.bn1 = nn.BatchNorm3d(channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv3d(channels, channels, kernel_size=3, stride=1,
                               padding=1, bias=False)
        self.bn2 = nn.BatchNorm3d(channels)
        self.downsample = downsample

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out

# 實例化 3D ResNet 模型
model = ResNet3D(BasicBlock, [3, 4, 6, 3], num_classes=3).to('cuda')



In [19]:
#model = Conv3DNet(96, 96, 96).to('cuda')
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 訓練3D CNN模型

此段代碼展示了如何訓練先前建立的3D CNN模型。

In [20]:
epochs = 10
best_val_loss = float('inf')
patience = 0
for epoch in range(epochs):
    # 訓練階段
    model.train()
    train_loss = 0.0
    all_train_preds = []
    all_train_targets = []
    for images, targets in tqdm(train_loader, desc=f"Training Epoch {epoch + 1}/{epochs}"):
        images = images.to('cuda')
        targets = targets.to('cuda')
        optimizer.zero_grad()
        outputs = model(images)
        loss = torch.nn.functional.cross_entropy(outputs, targets.long())
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

        # 收集預測和目標
        preds = torch.argmax(outputs, dim=1)
        all_train_preds.extend(preds.cpu().numpy())
        all_train_targets.extend(targets.cpu().numpy())

    # 驗證階段
    model.eval()
    val_loss = 0.0
    all_val_preds = []
    all_val_targets = []
    with torch.no_grad():
        for images, targets in tqdm(val_loader, desc=f"Validation Epoch {epoch + 1}/{epochs}"):
            images = images.to('cuda')
            targets = targets.to('cuda')
            outputs = model(images)
            loss = torch.nn.functional.cross_entropy(outputs, targets.long())
            val_loss += loss.item()

            # 收集預測和目標
            preds = torch.argmax(outputs, dim=1)
            all_val_preds.extend(preds.cpu().numpy())
            all_val_targets.extend(targets.cpu().numpy())

    # 計算準確度和 F1 分數
    train_accuracy = accuracy_score(all_train_targets, all_train_preds)
    train_f1 = f1_score(all_train_targets, all_train_preds, average='weighted')
    val_accuracy = accuracy_score(all_val_targets, all_val_preds)
    val_f1 = f1_score(all_val_targets, all_val_preds, average='weighted')

    print(f"Epoch {epoch+1}, Train Loss: {train_loss / len(train_loader)}, Validation Loss: {val_loss / len(val_loader)}")
    print(f"Training Accuracy: {train_accuracy}, Training F1: {train_f1}")
    print(f"Validation Accuracy: {val_accuracy}, Validation F1: {val_f1}")

    # 保存模型
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), 'best_model.pth')
        patience = 0
    else:
        patience += 1

    # 早期停止
    if patience > 3:
        print("Stopping early due to no improvement in validation loss.")
        break

Validation Epoch 1/10: 100%|██████████| 41/41 [00:11<00:00,  3.48it/s]


Epoch 1, Train Loss: 1.0634147644792713, Validation Loss: 1.4445612241582173
Training Accuracy: 0.4805194805194805, Training F1: 0.4365877257243443
Validation Accuracy: 0.14152410575427682, Validation F1: 0.03509180823879888


Validation Epoch 2/10: 100%|██████████| 41/41 [00:11<00:00,  3.59it/s]


Epoch 2, Train Loss: 0.9731184116699411, Validation Loss: 1.430737472162014
Training Accuracy: 0.5261707988980716, Training F1: 0.4947681008638093
Validation Accuracy: 0.24883359253499224, Validation F1: 0.21019713336976167


Validation Epoch 3/10: 100%|██████████| 41/41 [00:11<00:00,  3.63it/s]


...

Epoch 7, Train Loss: 0.778077818687607, Validation Loss: 2.8166375814414604
Training Accuracy: 0.6387249114521841, Training F1: 0.6273881666046895
Validation Accuracy: 0.18973561430793157, Validation F1: 0.12404811804872523


Validation Epoch 8/10: 100%|██████████| 41/41 [00:09<00:00,  4.16it/s]


Epoch 8, Train Loss: 0.7008895024938403, Validation Loss: 1.056671811313164
Training Accuracy: 0.6887052341597796, Training F1: 0.6827199861740928
Validation Accuracy: 0.42923794712286156, Validation F1: 0.4465480760284447


Validation Epoch 9/10: 100%|██████████| 41/41 [00:09<00:00,  4.14it/s]

Epoch 9, Train Loss: 0.6361420984912969, Validation Loss: 1.4946514149991477
Training Accuracy: 0.7221566312475404, Training F1: 0.7178940786032271
Validation Accuracy: 0.38724727838258166, Validation F1: 0.36856376668803176
Stopping early due to no improvement in validation loss.
